In [2]:
import pandas as pd
import numpy as np
import math

### Investigating Pearson Correlation Coefficients Using Paired T-Test with Hacker Statistics

We execute a two-sample permutation test to check statistical significance of the Pearson Correlation Coefficient between each population group and keyword. For easier reading and analysis, each keyword is given its own DataFrame, as described in the comments below.

In [3]:
""" dataframe key, for reference:

how to find number and routing number = kw_df_1
fiduciary bank = kw_df_2
foreign bank account = kw_df_3
balance your checkbook = kw_df_4
purchase money order = kw_df_5
savings account = kw_df_6
personal bankruptcy = kw_df_7
savings plan = kw_df_8
direct deposit = kw_df_9
529 plan = kw_df_10
credit card = kw_df_11
bankruptcy = kw_df_12
check cashing = kw_df_13
ATM = kw_df_14
fafsa = kw_df_15
savings association = kw_df_16
deposit money order = kw_df_17
deposit check = kw_df_18
best bank accounts = kw_df_19
small business bank = kw_df_20
"""

# extract data from cleaned csv: df
df = pd.read_csv('/Users/jacobschroeder/anaconda3/projects/test.csv')

# store population groups as a list: pop_list
pop_list = []
for i in range(101):
    pop_list.append("POP_" + str(i))

# pearson_r: Pearson Correlation Coefficient function
def pearson_r(x, y):
    """Compute Pearson correlation coefficient between two arrays."""
    # Compute correlation matrix: corr_mat
    corr_mat = np.corrcoef(x,y)

    # Return entry [0,1]
    return corr_mat[0,1]

# draw_bs_pairs: function to draw bootstrap pairs
def draw_bs_pairs(x, y, func, size=1):
    
    # Set up array of indices to sample from: inds
    inds = np.arange(len(x))

    # Initialize replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, len(inds))
        bs_replicates[i] = func(x[bs_inds], y[bs_inds])
        
    return bs_replicates

# function to remove unnecessary columns for this excercise: clean_dataframe
def clean_dataframe(df):
    del df['YEAR']
    del df['TOTAL_POP']
    del df['how to find number and routing number']
    del df['fiduciary bank']
    del df['foreign bank account']
    del df['balance your checkbook']
    del df['purchase money order']
    del df['savings account']
    del df['personal bankruptcy']
    del df['savings plan']
    del df['direct deposit']
    del df['529 plan']
    del df['credit card']
    del df['bankruptcy']
    del df['check cashing']
    del df['ATM']
    del df['fafsa']
    del df['savings association']
    del df['deposit money order']
    del df['deposit check']
    del df['best bank accounts']
    del df['small business bank']
    return df

In this two-sample t-test, we test randomly sampled Pearson Correlation Coefficients against the Pearson Correlation Coefficient extracted from the original dataset. 

The null hypothesis is that the correlation is no higher than what was initially sampled.

Testing assumes a 95% confidence interval, or 5% margin of error that the p-value will be evaluated against.

In [4]:
""" Our null hypothesis is that the selected keyword does not have
    a higher correlation to the age group than initially computed
"""

def test_pearson(kw,  i, size, pop_list):
    """ DOCSTRING: a function for testing pearson correlation coefficient over and over 
        kw: the keyword you wish to analyze, numpy array (independent variable)
        i: the starting row of the data (slice)
        size: the number of rows counted in the data analysis
    """
    # t: master counter
    t = 0
    return_list = np.empty(len(pop_list))
    return_pval = np.empty(len(pop_list))
    # for every age group in the population list
    for p in pop_list:
    
        # j, k: counters for operation
        j = 0
        k = 0

        # initialize empty numpy arrays to store data
        xdata = np.empty(size)
        ydata = np.empty(size)

        # xdata: an increase in a certain population group (independent) 
        for item in df[p][4:19]:
            xdata[j] = item
            j += 1
        j = 0

        # ydata: will create an increase in keyword demand (dependent)
        for item in df[kw][4:19]:
            ydata[k] = item
            k += 1
        k = 0

        # multiply each data set by 100 to ensure no "divide by zero" errors
        xdata = xdata
        ydata = ydata

        # run replicates test for selected age
        replicates = draw_bs_pairs(xdata, ydata, pearson_r, 10000)

        # get actual data for comparison
        actual_pearson = pearson_r(xdata, ydata)
        # print(actual_pearson)

        # p-value
        return_pval[t] = sum(abs(replicates) >= abs(actual_pearson)) / 10000
        
        return_list[t] = np.mean(replicates)
        print('Age Group {}: '.format(str(t)) + str(return_list[t]))
        print('P-Value: {}'.format(str(return_pval[t])))

        # increase master counter
        t += 1
    
    return { 'results': [return_list, return_pval] }

In [5]:
# setup the first dataframe
kw_df_1 = pd.DataFrame().reindex_like(df)
kw_df_1 = clean_dataframe(kw_df_1)
kw = 'how to find number and routing number'
kw_reps_1 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_1.columns)):
    kw_df_1.iloc[1, i] = kw_reps_1['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_1.columns)):
    kw_df_1.iloc[2, i] = kw_reps_1['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')
print(kw_df_1.head())

Age Group 0: 0.929201735296
P-Value: 0.675
Age Group 1: 0.936556583095
P-Value: 0.6727
Age Group 2: 0.944644606074
P-Value: 0.6739
Age Group 3: 0.955685210775
P-Value: 0.6889
Age Group 4: 0.930881680497
P-Value: 0.588
Age Group 5: 0.913379431824
P-Value: 0.6212
Age Group 6: 0.901809825105
P-Value: 0.6277
Age Group 7: 0.895777789166
P-Value: 0.6189
Age Group 8: 0.911204132691
P-Value: 0.5977
Age Group 9: 0.927846406959
P-Value: 0.6059
Age Group 10: 0.939168965035
P-Value: 0.6172
Age Group 11: 0.922704009332
P-Value: 0.5872
Age Group 12: 0.840816376623
P-Value: 0.5437
Age Group 13: 0.658497788439
P-Value: 0.5134
Age Group 14: 0.424785051551
P-Value: 0.503
Age Group 15: 0.246334496588
P-Value: 0.5464
Age Group 16: 0.101181445025
P-Value: 0.672
Age Group 17: -0.0116150129847
P-Value: 0.9087
Age Group 18: -0.141069584062
P-Value: 0.6374
Age Group 19: -0.0251386953203
P-Value: 0.9068
Age Group 20: 0.169218955104
P-Value: 0.6302
Age Group 21: 0.391096776461
P-Value: 0.5815
Age Group 22: 0.589

In [6]:
# setup dataframe 2
kw_df_2 = pd.DataFrame().reindex_like(df)
kw_df_2 = clean_dataframe(kw_df_2)
kw = 'fiduciary bank'
kw_reps_2 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_2.columns)):
    kw_df_2.iloc[1, i] = kw_reps_2['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_2.columns)):
    kw_df_2.iloc[2, i] = kw_reps_2['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_2.head())

Age Group 0: -0.0618532444766
P-Value: 0.8029
Age Group 1: -0.0669841694937
P-Value: 0.7863
Age Group 2: -0.0613884960843
P-Value: 0.7982
Age Group 3: -0.0231304288479
P-Value: 0.8964
Age Group 4: -0.0832339647825
P-Value: 0.7696
Age Group 5: -0.14468365966
P-Value: 0.6982
Age Group 6: -0.0939549786913
P-Value: 0.7555
Age Group 7: -0.0145821113811
P-Value: 0.8891
Age Group 8: 0.0506736713638
P-Value: 0.9348
Age Group 9: 0.0929287838567
P-Value: 0.792
Age Group 10: 0.0770214709396
P-Value: 0.7886
Age Group 11: 0.132463041391
P-Value: 0.5805
Age Group 12: 0.274457970098
P-Value: 0.5132
Age Group 13: 0.395505481213
P-Value: 0.5496
Age Group 14: 0.399444225054
P-Value: 0.5323
Age Group 15: 0.162263205027
P-Value: 0.6468
Age Group 16: -0.134233687293
P-Value: 0.6261
Age Group 17: -0.166526156627
P-Value: 0.6566
Age Group 18: -0.157951685606
P-Value: 0.6667
Age Group 19: -0.191480700313
P-Value: 0.6268
Age Group 20: -0.328157931876
P-Value: 0.5363
Age Group 21: -0.358696326467
P-Value: 0.527

In [7]:
# setup dataframe 3
kw_df_3 = pd.DataFrame().reindex_like(df)
kw_df_3 = clean_dataframe(kw_df_3)
kw = 'foreign bank account'
kw_reps_3 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_3.columns)):
    kw_df_3.iloc[1, i] = kw_reps_3['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_3.columns)):
    kw_df_3.iloc[2, i] = kw_reps_3['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_3.head())

Age Group 0: 0.83072851589
P-Value: 0.536
Age Group 1: 0.828821047136
P-Value: 0.5367
Age Group 2: 0.817639408665
P-Value: 0.5387
Age Group 3: 0.812077676853
P-Value: 0.5423
Age Group 4: 0.832921403848
P-Value: 0.5379
Age Group 5: 0.797279230693
P-Value: 0.5454
Age Group 6: 0.790718421602
P-Value: 0.5412
Age Group 7: 0.820625758689
P-Value: 0.5409
Age Group 8: 0.835257136101
P-Value: 0.5426
Age Group 9: 0.820338753257
P-Value: 0.5386
Age Group 10: 0.824185890312
P-Value: 0.5471
Age Group 11: 0.647788251507
P-Value: 0.5433
Age Group 12: 0.477806319755
P-Value: 0.5248
Age Group 13: 0.340340555505
P-Value: 0.5454
Age Group 14: 0.0785457361084
P-Value: 0.7685
Age Group 15: -0.0394779354625
P-Value: 0.9053
Age Group 16: -0.118814129265
P-Value: 0.7561
Age Group 17: -0.08357347363
P-Value: 0.8651
Age Group 18: 0.0590082752316
P-Value: 0.8077
Age Group 19: 0.320957692645
P-Value: 0.5246
Age Group 20: 0.483993000427
P-Value: 0.5347
Age Group 21: 0.608796272552
P-Value: 0.5459
Age Group 22: 0.6

In [8]:
# setup dataframe 4
kw_df_4 = pd.DataFrame().reindex_like(df)
kw_df_4 = clean_dataframe(kw_df_4)
kw = 'balance your checkbook'
kw_reps_4 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_4.columns)):
    kw_df_4.iloc[1, i] = kw_reps_4['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_4.columns)):
    kw_df_4.iloc[2, i] = kw_reps_4['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_4.head())

Age Group 0: 0.51095771845
P-Value: 0.5386
Age Group 1: 0.525961506467
P-Value: 0.5454
Age Group 2: 0.543452641393
P-Value: 0.5441
Age Group 3: 0.564405745227
P-Value: 0.5436
Age Group 4: 0.51065349835
P-Value: 0.5356
Age Group 5: 0.509853550578
P-Value: 0.5331
Age Group 6: 0.513495523406
P-Value: 0.5452
Age Group 7: 0.450240221195
P-Value: 0.5306
Age Group 8: 0.4815449916
P-Value: 0.5263
Age Group 9: 0.526624429862
P-Value: 0.5402
Age Group 10: 0.626594221252
P-Value: 0.5614
Age Group 11: 0.684141334529
P-Value: 0.5736
Age Group 12: 0.7408490012
P-Value: 0.5847
Age Group 13: 0.725007408838
P-Value: 0.5736
Age Group 14: 0.613313013983
P-Value: 0.5383
Age Group 15: 0.41486424721
P-Value: 0.5027
Age Group 16: 0.254874303258
P-Value: 0.569
Age Group 17: 0.0563488778567
P-Value: 0.7566
Age Group 18: -0.306480209082
P-Value: 0.5897
Age Group 19: -0.344230834454
P-Value: 0.586
Age Group 20: -0.265243939062
P-Value: 0.5606
Age Group 21: -0.0292108119658
P-Value: 0.8259
Age Group 22: 0.1582696

In [9]:
# setup dataframe 5
kw_df_5 = pd.DataFrame().reindex_like(df)
kw_df_5 = clean_dataframe(kw_df_5)
kw = 'purchase money order'
kw_reps_5 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_5.columns)):
    kw_df_5.iloc[1, i] = kw_reps_5['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_5.columns)):
    kw_df_5.iloc[2, i] = kw_reps_5['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_5.head())

Age Group 0: 0.705626075892
P-Value: 0.553
Age Group 1: 0.71958560455
P-Value: 0.5543
Age Group 2: 0.724765207047
P-Value: 0.547
Age Group 3: 0.741238028457
P-Value: 0.5573
Age Group 4: 0.706173794129
P-Value: 0.5643
Age Group 5: 0.668934601741
P-Value: 0.5574
Age Group 6: 0.632600673617
P-Value: 0.5673
Age Group 7: 0.626937214513
P-Value: 0.5564
Age Group 8: 0.680803892676
P-Value: 0.5489
Age Group 9: 0.762012103572
P-Value: 0.5476
Age Group 10: 0.881781091819
P-Value: 0.5235
Age Group 11: 0.90002874682
P-Value: 0.5201
Age Group 12: 0.887831273973
P-Value: 0.5412
Age Group 13: 0.797916380847
P-Value: 0.5356
Age Group 14: 0.648390081764
P-Value: 0.5326
Age Group 15: 0.410351482567
P-Value: 0.5159
Age Group 16: 0.0909814842677
P-Value: 0.7086
Age Group 17: -0.28618777138
P-Value: 0.5636
Age Group 18: -0.505571055207
P-Value: 0.5541
Age Group 19: -0.368409992033
P-Value: 0.5272
Age Group 20: -0.108723103379
P-Value: 0.6071
Age Group 21: 0.168552678482
P-Value: 0.5476
Age Group 22: 0.3632

In [10]:
# setup dataframe 6
kw_df_6 = pd.DataFrame().reindex_like(df)
kw_df_6 = clean_dataframe(kw_df_6)
kw = 'savings account'
kw_reps_6 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_6.columns)):
    kw_df_6.iloc[1, i] = kw_reps_6['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_6.columns)):
    kw_df_6.iloc[2, i] = kw_reps_6['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_6.head())

Age Group 0: 0.726181418716
P-Value: 0.5892
Age Group 1: 0.739671584878
P-Value: 0.6012
Age Group 2: 0.757787214975
P-Value: 0.6017
Age Group 3: 0.77631073621
P-Value: 0.6062
Age Group 4: 0.771532263867
P-Value: 0.6236
Age Group 5: 0.774121107232
P-Value: 0.6472
Age Group 6: 0.769590014629
P-Value: 0.6508
Age Group 7: 0.742797895294
P-Value: 0.6072
Age Group 8: 0.712964594485
P-Value: 0.5839
Age Group 9: 0.674680662861
P-Value: 0.5575
Age Group 10: 0.664538092554
P-Value: 0.5475
Age Group 11: 0.659413758343
P-Value: 0.5456
Age Group 12: 0.654536627046
P-Value: 0.5366
Age Group 13: 0.611774642497
P-Value: 0.5275
Age Group 14: 0.518363617115
P-Value: 0.5048
Age Group 15: 0.48435974914
P-Value: 0.516
Age Group 16: 0.495682956914
P-Value: 0.5061
Age Group 17: 0.372868321697
P-Value: 0.4868
Age Group 18: 0.0984198659682
P-Value: 0.673
Age Group 19: -0.00302242830186
P-Value: 0.9773
Age Group 20: -0.0160427067539
P-Value: 0.9296
Age Group 21: 0.0999637499628
P-Value: 0.7502
Age Group 22: 0.2

In [11]:
# setup dataframe 7
kw_df_7 = pd.DataFrame().reindex_like(df)
kw_df_7 = clean_dataframe(kw_df_7)
kw = 'personal bankruptcy'
kw_reps_7 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_7.columns)):
    kw_df_7.iloc[1, i] = kw_reps_7['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_7.columns)):
    kw_df_7.iloc[2, i] = kw_reps_7['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_7.head())

Age Group 0: -0.878001580627
P-Value: 0.5531
Age Group 1: -0.88136311091
P-Value: 0.5574
Age Group 2: -0.883763556937
P-Value: 0.557
Age Group 3: -0.862953498461
P-Value: 0.5727
Age Group 4: -0.868244141658
P-Value: 0.5683
Age Group 5: -0.903144458143
P-Value: 0.5474
Age Group 6: -0.884170216907
P-Value: 0.5349
Age Group 7: -0.846001014213
P-Value: 0.5244
Age Group 8: -0.795224998017
P-Value: 0.5656
Age Group 9: -0.785334678933
P-Value: 0.6238
Age Group 10: -0.760568105697
P-Value: 0.6193
Age Group 11: -0.684427766227
P-Value: 0.5721
Age Group 12: -0.516876856582
P-Value: 0.5501
Age Group 13: -0.2505164049
P-Value: 0.5749
Age Group 14: 0.0169308682436
P-Value: 0.8844
Age Group 15: 0.0784156122251
P-Value: 0.7525
Age Group 16: -0.0146776560742
P-Value: 0.8835
Age Group 17: -0.0673893563871
P-Value: 0.7436
Age Group 18: -0.0656889582523
P-Value: 0.7552
Age Group 19: -0.214255633362
P-Value: 0.6126
Age Group 20: -0.463519448656
P-Value: 0.5147
Age Group 21: -0.671019634826
P-Value: 0.5255

In [12]:
# setup dataframe 8
kw_df_8 = pd.DataFrame().reindex_like(df)
kw_df_8 = clean_dataframe(kw_df_8)
kw = 'savings plan'
kw_reps_8 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_8.columns)):
    kw_df_8.iloc[1, i] = kw_reps_8['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_8.columns)):
    kw_df_8.iloc[2, i] = kw_reps_8['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_8.head())

Age Group 0: -0.857219381884
P-Value: 0.5308
Age Group 1: -0.848682178312
P-Value: 0.5358
Age Group 2: -0.837478509135
P-Value: 0.5372
Age Group 3: -0.821639552201
P-Value: 0.5292
Age Group 4: -0.830994006393
P-Value: 0.5365
Age Group 5: -0.83485439041
P-Value: 0.5259
Age Group 6: -0.844243476775
P-Value: 0.5485
Age Group 7: -0.869138541911
P-Value: 0.5352
Age Group 8: -0.847156010404
P-Value: 0.5356
Age Group 9: -0.793220321155
P-Value: 0.5538
Age Group 10: -0.71325101472
P-Value: 0.5444
Age Group 11: -0.559359909708
P-Value: 0.5457
Age Group 12: -0.332143792571
P-Value: 0.5282
Age Group 13: -0.0565398294918
P-Value: 0.8244
Age Group 14: 0.222368380294
P-Value: 0.5221
Age Group 15: 0.279769933786
P-Value: 0.5254
Age Group 16: 0.254369984624
P-Value: 0.5823
Age Group 17: 0.0908960048606
P-Value: 0.8618
Age Group 18: -0.216844053131
P-Value: 0.5886
Age Group 19: -0.484340702608
P-Value: 0.5473
Age Group 20: -0.705013288392
P-Value: 0.5512
Age Group 21: -0.820047694371
P-Value: 0.5795
Ag

In [13]:
# setup dataframe 9
kw_df_9 = pd.DataFrame().reindex_like(df)
kw_df_9 = clean_dataframe(kw_df_9)
kw = 'direct deposit'
kw_reps_9 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_9.columns)):
    kw_df_9.iloc[1, i] = kw_reps_9['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_9.columns)):
    kw_df_9.iloc[2, i] = kw_reps_9['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_9.head())

Age Group 0: 0.97562471048
P-Value: 0.6073
Age Group 1: 0.97631353973
P-Value: 0.6263
Age Group 2: 0.976678240822
P-Value: 0.6488
Age Group 3: 0.978436842556
P-Value: 0.6823
Age Group 4: 0.970287030892
P-Value: 0.6896
Age Group 5: 0.959764824683
P-Value: 0.7457
Age Group 6: 0.958915389304
P-Value: 0.6761
Age Group 7: 0.96911413588
P-Value: 0.65
Age Group 8: 0.961291656872
P-Value: 0.6577
Age Group 9: 0.949868166611
P-Value: 0.6556
Age Group 10: 0.921083936106
P-Value: 0.6394
Age Group 11: 0.836561787097
P-Value: 0.5703
Age Group 12: 0.689216137439
P-Value: 0.5468
Age Group 13: 0.454407533915
P-Value: 0.5261
Age Group 14: 0.190126656587
P-Value: 0.5966
Age Group 15: 0.0507531606248
P-Value: 0.7983
Age Group 16: 0.00614433143458
P-Value: 0.8798
Age Group 17: 0.0717659425778
P-Value: 0.7407
Age Group 18: 0.0556170446873
P-Value: 0.791
Age Group 19: 0.221792190282
P-Value: 0.5941
Age Group 20: 0.407146242364
P-Value: 0.5687
Age Group 21: 0.583838047517
P-Value: 0.5694
Age Group 22: 0.73682

In [14]:
# setup dataframe 10
kw_df_10 = pd.DataFrame().reindex_like(df)
kw_df_10 = clean_dataframe(kw_df_10)
kw = '529 plan'
kw_reps_10 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_10.columns)):
    kw_df_10.iloc[1, i] = kw_reps_10['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_10.columns)):
    kw_df_10.iloc[2, i] = kw_reps_10['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_10.head())

Age Group 0: -0.457249953656
P-Value: 0.5312
Age Group 1: -0.43836534667
P-Value: 0.525
Age Group 2: -0.41690446014
P-Value: 0.5244
Age Group 3: -0.391189527062
P-Value: 0.5249
Age Group 4: -0.410349590707
P-Value: 0.522
Age Group 5: -0.39930864943
P-Value: 0.5329
Age Group 6: -0.415237208931
P-Value: 0.5333
Age Group 7: -0.462142439862
P-Value: 0.5247
Age Group 8: -0.458834277172
P-Value: 0.5203
Age Group 9: -0.430131940768
P-Value: 0.5188
Age Group 10: -0.352227788696
P-Value: 0.5444
Age Group 11: -0.173696331423
P-Value: 0.7044
Age Group 12: 0.0284447758431
P-Value: 0.8674
Age Group 13: 0.275915751719
P-Value: 0.5142
Age Group 14: 0.486888881984
P-Value: 0.5103
Age Group 15: 0.482735560641
P-Value: 0.5153
Age Group 16: 0.516982763271
P-Value: 0.5395
Age Group 17: 0.326051976037
P-Value: 0.5578
Age Group 18: -0.168406547496
P-Value: 0.5613
Age Group 19: -0.496682165704
P-Value: 0.546
Age Group 20: -0.715105216772
P-Value: 0.5456
Age Group 21: -0.740947332518
P-Value: 0.575
Age Group 

In [15]:
# setup dataframe 11
kw_df_11 = pd.DataFrame().reindex_like(df)
kw_df_11 = clean_dataframe(kw_df_11)
kw = 'credit card'
kw_reps_11 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_11.columns)):
    kw_df_11.iloc[1, i] = kw_reps_11['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_11.columns)):
    kw_df_11.iloc[2, i] = kw_reps_11['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_11.head())

Age Group 0: 0.975538757002
P-Value: 0.5704
Age Group 1: 0.978819778657
P-Value: 0.5902
Age Group 2: 0.982796780502
P-Value: 0.5798
Age Group 3: 0.989408525157
P-Value: 0.5634
Age Group 4: 0.967993462843
P-Value: 0.552
Age Group 5: 0.955414974209
P-Value: 0.6048
Age Group 6: 0.947880158966
P-Value: 0.5882
Age Group 7: 0.951617097278
P-Value: 0.5656
Age Group 8: 0.962426653831
P-Value: 0.5206
Age Group 9: 0.953087054219
P-Value: 0.5437
Age Group 10: 0.947382007505
P-Value: 0.5671
Age Group 11: 0.887661540875
P-Value: 0.5696
Age Group 12: 0.773000740491
P-Value: 0.542
Age Group 13: 0.584801164774
P-Value: 0.5292
Age Group 14: 0.311231291615
P-Value: 0.5401
Age Group 15: 0.0989832424287
P-Value: 0.7024
Age Group 16: -0.00714268248682
P-Value: 0.9511
Age Group 17: -0.0435000364081
P-Value: 0.9511
Age Group 18: -0.0658611086512
P-Value: 0.8431
Age Group 19: 0.107765451726
P-Value: 0.7201
Age Group 20: 0.299572099861
P-Value: 0.5736
Age Group 21: 0.509464892048
P-Value: 0.5559
Age Group 22: 

In [16]:
# setup dataframe 12
kw_df_12 = pd.DataFrame().reindex_like(df)
kw_df_12 = clean_dataframe(kw_df_12)
kw = 'bankruptcy'
kw_reps_12 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_12.columns)):
    kw_df_12.iloc[1, i] = kw_reps_12['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_12.columns)):
    kw_df_12.iloc[2, i] = kw_reps_12['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_12.head())

Age Group 0: -0.731579313362
P-Value: 0.5568
Age Group 1: -0.745098875473
P-Value: 0.5625
Age Group 2: -0.758442563535
P-Value: 0.5674
Age Group 3: -0.752094842636
P-Value: 0.5687
Age Group 4: -0.723496837322
P-Value: 0.5788
Age Group 5: -0.743444535445
P-Value: 0.5668
Age Group 6: -0.703446704768
P-Value: 0.5683
Age Group 7: -0.654378285756
P-Value: 0.5525
Age Group 8: -0.631637521947
P-Value: 0.5565
Age Group 9: -0.691311680796
P-Value: 0.5382
Age Group 10: -0.750926879419
P-Value: 0.5405
Age Group 11: -0.797682501181
P-Value: 0.554
Age Group 12: -0.752493508472
P-Value: 0.545
Age Group 13: -0.568945971525
P-Value: 0.5335
Age Group 14: -0.335445039896
P-Value: 0.5193
Age Group 15: -0.183532660727
P-Value: 0.5939
Age Group 16: -0.0649230548777
P-Value: 0.7983
Age Group 17: 0.104668277532
P-Value: 0.7406
Age Group 18: 0.306132979332
P-Value: 0.5487
Age Group 19: 0.238439076651
P-Value: 0.5727
Age Group 20: -0.0412606535995
P-Value: 0.8493
Age Group 21: -0.325809002632
P-Value: 0.5095
A

In [17]:
# setup dataframe 13
kw_df_13 = pd.DataFrame().reindex_like(df)
kw_df_13 = clean_dataframe(kw_df_13)
kw = 'check cashing'
kw_reps_13 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_13.columns)):
    kw_df_13.iloc[1, i] = kw_reps_13['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_13.columns)):
    kw_df_13.iloc[2, i] = kw_reps_13['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_13.head())

Age Group 0: 0.97693794024
P-Value: 0.6187
Age Group 1: 0.976646785509
P-Value: 0.6233
Age Group 2: 0.975766230588
P-Value: 0.6165
Age Group 3: 0.977619849294
P-Value: 0.6093
Age Group 4: 0.959410098313
P-Value: 0.6131
Age Group 5: 0.946302353049
P-Value: 0.6544
Age Group 6: 0.94238654257
P-Value: 0.6184
Age Group 7: 0.957773420332
P-Value: 0.5928
Age Group 8: 0.965665520592
P-Value: 0.5601
Age Group 9: 0.960745327863
P-Value: 0.5751
Age Group 10: 0.950862424543
P-Value: 0.6182
Age Group 11: 0.868140023358
P-Value: 0.5811
Age Group 12: 0.713707778387
P-Value: 0.5491
Age Group 13: 0.461173433611
P-Value: 0.5298
Age Group 14: 0.184191649185
P-Value: 0.594
Age Group 15: 0.0303158312891
P-Value: 0.8715
Age Group 16: -0.0744416431477
P-Value: 0.8638
Age Group 17: -0.0888268788414
P-Value: 0.8161
Age Group 18: -0.0373331885239
P-Value: 0.9074
Age Group 19: 0.167010850565
P-Value: 0.6418
Age Group 20: 0.406094289968
P-Value: 0.5575
Age Group 21: 0.622042164799
P-Value: 0.5653
Age Group 22: 0.

In [18]:
# setup dataframe 14
kw_df_14 = pd.DataFrame().reindex_like(df)
kw_df_14 = clean_dataframe(kw_df_14)
kw = 'ATM'
kw_reps_14 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_14.columns)):
    kw_df_14.iloc[1, i] = kw_reps_14['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_14.columns)):
    kw_df_14.iloc[2, i] = kw_reps_14['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_14.head())

Age Group 0: 0.936714407321
P-Value: 0.5931
Age Group 1: 0.940077930158
P-Value: 0.5844
Age Group 2: 0.94452216537
P-Value: 0.5869
Age Group 3: 0.954690091178
P-Value: 0.5842
Age Group 4: 0.924375806828
P-Value: 0.57
Age Group 5: 0.902542058311
P-Value: 0.5925
Age Group 6: 0.8956580571
P-Value: 0.5967
Age Group 7: 0.904377517442
P-Value: 0.5911
Age Group 8: 0.926621839137
P-Value: 0.5616
Age Group 9: 0.951504866127
P-Value: 0.5659
Age Group 10: 0.969776749232
P-Value: 0.6256
Age Group 11: 0.934563742878
P-Value: 0.5749
Age Group 12: 0.81366734068
P-Value: 0.5506
Age Group 13: 0.616215390091
P-Value: 0.5244
Age Group 14: 0.363187584548
P-Value: 0.5171
Age Group 15: 0.161454940707
P-Value: 0.6223
Age Group 16: 0.000885617078328
P-Value: 0.8872
Age Group 17: -0.114371447907
P-Value: 0.8179
Age Group 18: -0.172417785536
P-Value: 0.6004
Age Group 19: 0.00460644726746
P-Value: 0.9999
Age Group 20: 0.238032045877
P-Value: 0.5845
Age Group 21: 0.477441293211
P-Value: 0.581
Age Group 22: 0.6597

In [19]:
# setup dataframe 15
kw_df_15 = pd.DataFrame().reindex_like(df)
kw_df_15 = clean_dataframe(kw_df_15)
kw = 'fafsa'
kw_reps_15 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_15.columns)):
    kw_df_15.iloc[1, i] = kw_reps_15['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_15.columns)):
    kw_df_15.iloc[2, i] = kw_reps_15['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_15.head())

Age Group 0: -0.891503297327
P-Value: 0.538
Age Group 1: -0.88540115558
P-Value: 0.5388
Age Group 2: -0.884122320953
P-Value: 0.5384
Age Group 3: -0.887780864278
P-Value: 0.5373
Age Group 4: -0.873306973823
P-Value: 0.5626
Age Group 5: -0.879107829672
P-Value: 0.56
Age Group 6: -0.886966585916
P-Value: 0.5498
Age Group 7: -0.901792020027
P-Value: 0.5501
Age Group 8: -0.913623131337
P-Value: 0.5477
Age Group 9: -0.886226285751
P-Value: 0.5413
Age Group 10: -0.752203327168
P-Value: 0.5367
Age Group 11: -0.665192176324
P-Value: 0.5405
Age Group 12: -0.533100378389
P-Value: 0.5443
Age Group 13: -0.304802261798
P-Value: 0.5399
Age Group 14: -0.0770666694742
P-Value: 0.7741
Age Group 15: 0.0863473925076
P-Value: 0.7861
Age Group 16: 0.0939086900176
P-Value: 0.7824
Age Group 17: -0.0247068410015
P-Value: 0.8889
Age Group 18: -0.181512671693
P-Value: 0.6023
Age Group 19: -0.375155414607
P-Value: 0.5323
Age Group 20: -0.520032686134
P-Value: 0.5513
Age Group 21: -0.586861197929
P-Value: 0.5449


In [20]:
# setup dataframe 16
kw_df_16 = pd.DataFrame().reindex_like(df)
kw_df_16 = clean_dataframe(kw_df_16)
kw = 'savings association'
kw_reps_16 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_16.columns)):
    kw_df_16.iloc[1, i] = kw_reps_16['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_16.columns)):
    kw_df_16.iloc[2, i] = kw_reps_16['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_16.head())

Age Group 0: -0.946302886707
P-Value: 0.626
Age Group 1: -0.94239167964
P-Value: 0.6178
Age Group 2: -0.935497391956
P-Value: 0.6208
Age Group 3: -0.918570459399
P-Value: 0.643
Age Group 4: -0.934419535197
P-Value: 0.5784
Age Group 5: -0.94385261878
P-Value: 0.5746
Age Group 6: -0.946965975066
P-Value: 0.5874
Age Group 7: -0.942506576477
P-Value: 0.6282
Age Group 8: -0.912854109731
P-Value: 0.6268
Age Group 9: -0.864241952325
P-Value: 0.62
Age Group 10: -0.785588704224
P-Value: 0.613
Age Group 11: -0.657623059394
P-Value: 0.5754
Age Group 12: -0.445966671417
P-Value: 0.5593
Age Group 13: -0.170826852677
P-Value: 0.6644
Age Group 14: 0.11778314415
P-Value: 0.6298
Age Group 15: 0.200111121465
P-Value: 0.554
Age Group 16: 0.130698579776
P-Value: 0.7565
Age Group 17: -0.0300127628169
P-Value: 0.8026
Age Group 18: -0.208219046752
P-Value: 0.6047
Age Group 19: -0.43743014723
P-Value: 0.5378
Age Group 20: -0.647567469822
P-Value: 0.537
Age Group 21: -0.787226413138
P-Value: 0.5724
Age Group 2

In [21]:
# setup dataframe 17
kw_df_17 = pd.DataFrame().reindex_like(df)
kw_df_17 = clean_dataframe(kw_df_17)
kw = 'deposit money order'
kw_reps_17 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_17.columns)):
    kw_df_17.iloc[1, i] = kw_reps_17['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_17.columns)):
    kw_df_17.iloc[2, i] = kw_reps_17['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_17.head())

Age Group 0: 0.93077641943
P-Value: 0.6251
Age Group 1: 0.938756875248
P-Value: 0.63
Age Group 2: 0.946164463819
P-Value: 0.6282
Age Group 3: 0.956604612173
P-Value: 0.6266
Age Group 4: 0.931793161682
P-Value: 0.582
Age Group 5: 0.909396134415
P-Value: 0.5954
Age Group 6: 0.895792535825
P-Value: 0.6035
Age Group 7: 0.894021941725
P-Value: 0.5843
Age Group 8: 0.908541520997
P-Value: 0.5684
Age Group 9: 0.930723278112
P-Value: 0.5608
Age Group 10: 0.945709484244
P-Value: 0.5785
Age Group 11: 0.937775090136
P-Value: 0.579
Age Group 12: 0.861768989572
P-Value: 0.5378
Age Group 13: 0.679548268522
P-Value: 0.5238
Age Group 14: 0.445530951304
P-Value: 0.518
Age Group 15: 0.23568397423
P-Value: 0.5632
Age Group 16: 0.0668882367557
P-Value: 0.7372
Age Group 17: -0.05991460916
P-Value: 0.9741
Age Group 18: -0.176380727499
P-Value: 0.5883
Age Group 19: -0.0454306672445
P-Value: 0.8277
Age Group 20: 0.155582183733
P-Value: 0.6366
Age Group 21: 0.38885381469
P-Value: 0.5682
Age Group 22: 0.60455391

In [22]:
# setup dataframe 18
kw_df_18 = pd.DataFrame().reindex_like(df)
kw_df_18 = clean_dataframe(kw_df_18)
kw = 'deposit check'
kw_reps_18 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_18.columns)):
    kw_df_18.iloc[1, i] = kw_reps_18['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_18.columns)):
    kw_df_18.iloc[2, i] = kw_reps_18['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_18.head())

Age Group 0: 0.981757775925
P-Value: 0.5944
Age Group 1: 0.983638567725
P-Value: 0.6002
Age Group 2: 0.984934388688
P-Value: 0.5922
Age Group 3: 0.988079198353
P-Value: 0.5724
Age Group 4: 0.965747643592
P-Value: 0.573
Age Group 5: 0.951011540284
P-Value: 0.6024
Age Group 6: 0.94572273397
P-Value: 0.5987
Age Group 7: 0.953385888029
P-Value: 0.5544
Age Group 8: 0.961412956114
P-Value: 0.5353
Age Group 9: 0.967648595021
P-Value: 0.5444
Age Group 10: 0.96325790206
P-Value: 0.5888
Age Group 11: 0.908161314457
P-Value: 0.5597
Age Group 12: 0.770702834307
P-Value: 0.54
Age Group 13: 0.527240904169
P-Value: 0.5239
Age Group 14: 0.244965828008
P-Value: 0.5572
Age Group 15: 0.0456274815568
P-Value: 0.8171
Age Group 16: -0.0599841503659
P-Value: 0.9132
Age Group 17: -0.0899883439059
P-Value: 0.8577
Age Group 18: -0.0847679629271
P-Value: 0.7792
Age Group 19: 0.107978529085
P-Value: 0.7213
Age Group 20: 0.344896419488
P-Value: 0.5552
Age Group 21: 0.578462703805
P-Value: 0.574
Age Group 22: 0.762

In [23]:
# setup dataframe 19
kw_df_19 = pd.DataFrame().reindex_like(df)
kw_df_19 = clean_dataframe(kw_df_19)
kw = 'best bank accounts'
kw_reps_19 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_19.columns)):
    kw_df_19.iloc[1, i] = kw_reps_19['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_19.columns)):
    kw_df_19.iloc[2, i] = kw_reps_19['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_19.head())

Age Group 0: 0.937370616543
P-Value: 0.6384
Age Group 1: 0.942621751399
P-Value: 0.6423
Age Group 2: 0.950603963784
P-Value: 0.6475
Age Group 3: 0.957599293907
P-Value: 0.6444
Age Group 4: 0.94972516502
P-Value: 0.7033
Age Group 5: 0.948120169037
P-Value: 0.7168
Age Group 6: 0.939090228342
P-Value: 0.6878
Age Group 7: 0.94195218346
P-Value: 0.6534
Age Group 8: 0.918002566978
P-Value: 0.6179
Age Group 9: 0.89519735373
P-Value: 0.5967
Age Group 10: 0.869655826144
P-Value: 0.5774
Age Group 11: 0.811207970152
P-Value: 0.5783
Age Group 12: 0.713282146794
P-Value: 0.5493
Age Group 13: 0.52884879829
P-Value: 0.5147
Age Group 14: 0.316614681892
P-Value: 0.5437
Age Group 15: 0.204468679925
P-Value: 0.5967
Age Group 16: 0.191234504455
P-Value: 0.596
Age Group 17: 0.150686242773
P-Value: 0.5996
Age Group 18: 0.0767010731478
P-Value: 0.7437
Age Group 19: 0.143376839602
P-Value: 0.6859
Age Group 20: 0.265786070458
P-Value: 0.5765
Age Group 21: 0.437140889868
P-Value: 0.5746
Age Group 22: 0.59640320

In [24]:
# setup dataframe 20
kw_df_20 = pd.DataFrame().reindex_like(df)
kw_df_20 = clean_dataframe(kw_df_20)
kw = 'small business bank'
kw_reps_20 = test_pearson(kw, 4, 15, pop_list)
print('Permutation Test Complete')

# place corresponding permuatation values in first row of dataframe
for i in range(len(kw_df_20.columns)):
    kw_df_20.iloc[1, i] = kw_reps_20['results'][0][i]
    print('success 1: ' + str(i))
print('Checkpoint 2')

# p-value loading in the second row
for i in range(len(kw_df_20.columns)):
    kw_df_20.iloc[2, i] = kw_reps_20['results'][1][i]
    print('success 2: ' + str(i))
print('Checkpoint 3')

print('Complete')

print(kw_df_20.head())

Age Group 0: 0.829180605129
P-Value: 0.5274
Age Group 1: 0.834563390213
P-Value: 0.5191
Age Group 2: 0.832641008014
P-Value: 0.5154
Age Group 3: 0.826092029641
P-Value: 0.5108
Age Group 4: 0.859657826194
P-Value: 0.5204
Age Group 5: 0.842827517463
P-Value: 0.5209
Age Group 6: 0.839156939286
P-Value: 0.527
Age Group 7: 0.823373980487
P-Value: 0.5286
Age Group 8: 0.815441849534
P-Value: 0.5332
Age Group 9: 0.76927081723
P-Value: 0.5329
Age Group 10: 0.75459200167
P-Value: 0.5456
Age Group 11: 0.656368135581
P-Value: 0.548
Age Group 12: 0.542069958796
P-Value: 0.5436
Age Group 13: 0.336836992582
P-Value: 0.5648
Age Group 14: 0.140647431268
P-Value: 0.6718
Age Group 15: 0.13105076859
P-Value: 0.6288
Age Group 16: 0.24075601418
P-Value: 0.558
Age Group 17: 0.250082227963
P-Value: 0.5522
Age Group 18: 0.179556318602
P-Value: 0.6155
Age Group 19: 0.257259741641
P-Value: 0.5741
Age Group 20: 0.345577001308
P-Value: 0.5547
Age Group 21: 0.480309321535
P-Value: 0.5707
Age Group 22: 0.57803399446

In [25]:
kw_df_1.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_1.csv')
kw_df_2.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_2.csv')
kw_df_3.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_3.csv')
kw_df_4.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_4.csv')
kw_df_5.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_5.csv')
kw_df_6.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_6.csv')
kw_df_7.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_7.csv')
kw_df_8.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_8.csv')
kw_df_9.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_9.csv')
kw_df_10.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_10.csv')
kw_df_11.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_11.csv')
kw_df_12.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_12.csv')
kw_df_13.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_13.csv')
kw_df_14.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_14.csv')
kw_df_15.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_15.csv')
kw_df_16.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_16.csv')
kw_df_17.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_17.csv')
kw_df_18.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_18.csv')
kw_df_19.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_19.csv')
kw_df_20.to_csv('/Users/jacobschroeder/anaconda3/projects/kw_df_20.csv')